# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.87/6560/1 Dashboard: http://10.1.0.87:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1006,Michael,0.4438237483732903,-0.16497353011948368
2000-01-01 00:00:01,977,Ray,-0.9155987708886735,-0.17905314486246127
2000-01-01 00:00:02,1017,Michael,0.5992153861163148,0.2214672646361151
2000-01-01 00:00:03,1011,Ray,0.9989053554511005,0.810811038465912
2000-01-01 00:00:04,1011,Michael,-0.5010431330428871,0.7170121070291564
2000-01-01 00:00:05,1004,Victor,-0.6636719996034222,0.7714996991959673
2000-01-01 00:00:06,971,Ingrid,-0.02119865509250496,0.15397319121009145
2000-01-01 00:00:07,1018,Ray,-0.6315003961441854,0.2451524825722491
2000-01-01 00:00:08,1003,Ursula,0.9275484123478988,-0.932081253370096


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1034,Norbert,0.06651169542295454,-0.8565767673830691
2000-01-30 00:00:01,1004,Xavier,0.9542671220943273,-0.9368543036285701
2000-01-30 00:00:02,988,Hannah,-0.9365015907645551,-0.7861468417638435
2000-01-30 00:00:03,988,Sarah,0.20532419454311168,0.39375310518055806
2000-01-30 00:00:04,1007,Zelda,-0.47406471330966804,0.6623427605288377
2000-01-30 00:00:05,931,George,-0.6225131878425527,0.501329715851671
2000-01-30 00:00:06,1064,Dan,0.2569658948830271,-0.6128835677743929
2000-01-30 00:00:07,1002,Laura,0.1492286428419194,-0.5174069488535107
2000-01-30 00:00:08,984,Bob,-0.8305211797333942,0.922411455167149


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1006,Michael,0.443824,-0.164974
1,2000-01-01 00:00:01,977,Ray,-0.915599,-0.179053
2,2000-01-01 00:00:02,1017,Michael,0.599215,0.221467
3,2000-01-01 00:00:03,1011,Ray,0.998905,0.810811
4,2000-01-01 00:00:04,1011,Michael,-0.501043,0.717012


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1006,Michael,0.443824,-0.164974
1,2000-01-01 00:00:01,977,Ray,-0.915599,-0.179053
2,2000-01-01 00:00:02,1017,Michael,0.599215,0.221467
3,2000-01-01 00:00:03,1011,Ray,0.998905,0.810811
4,2000-01-01 00:00:04,1011,Michael,-0.501043,0.717012


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.32 s, sys: 394 ms, total: 3.72 s
Wall time: 3.82 s


name
Alice      -0.002458
Bob        -0.000990
Charlie    -0.001645
Dan         0.001700
Edith      -0.000237
Frank       0.001324
George      0.000356
Hannah      0.002240
Ingrid      0.002844
Jerry       0.001827
Kevin      -0.002894
Laura      -0.001119
Michael    -0.003905
Norbert    -0.002496
Oliver     -0.002869
Patricia   -0.001616
Quinn       0.001298
Ray         0.000411
Sarah       0.001070
Tim         0.006105
Ursula     -0.000201
Victor      0.000647
Wendy      -0.000604
Xavier     -0.001101
Yvonne     -0.000980
Zelda       0.001932
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.11 s, sys: 104 ms, total: 1.21 s
Wall time: 1.28 s


name
Alice      -0.002458
Bob        -0.000990
Charlie    -0.001645
Dan         0.001700
Edith      -0.000237
Frank       0.001324
George      0.000356
Hannah      0.002240
Ingrid      0.002844
Jerry       0.001827
Kevin      -0.002894
Laura      -0.001119
Michael    -0.003905
Norbert    -0.002496
Oliver     -0.002869
Patricia   -0.001616
Quinn       0.001298
Ray         0.000411
Sarah       0.001070
Tim         0.006105
Ursula     -0.000201
Victor      0.000647
Wendy      -0.000604
Xavier     -0.001101
Yvonne     -0.000980
Zelda       0.001932
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 908 ms, sys: 78.8 ms, total: 987 ms
Wall time: 1.13 s


name
Alice      -0.002458
Bob        -0.000990
Charlie    -0.001645
Dan         0.001700
Edith      -0.000237
Frank       0.001324
George      0.000356
Hannah      0.002240
Ingrid      0.002844
Jerry       0.001827
Kevin      -0.002894
Laura      -0.001119
Michael    -0.003905
Norbert    -0.002496
Oliver     -0.002869
Patricia   -0.001616
Quinn       0.001298
Ray         0.000411
Sarah       0.001070
Tim         0.006105
Ursula     -0.000201
Victor      0.000647
Wendy      -0.000604
Xavier     -0.001101
Yvonne     -0.000980
Zelda       0.001932
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html